In [1]:
import matplotlib
matplotlib.use('pdf')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
plt.style.use('default')

In [3]:
def human_format(x):
    sigpart = 0
    suffix = ""
    if x < 1000:
        return "{:.3g}".format(x)
    elif x < 1000000:
        sigpart = x / 1000
        suffix = "k"
    elif x < 1000000000:
        sigpart = x / 1000000
        suffix = "M"
    else:
        sigpart = x / 1000000000
        suffix = "B"
    return "{:.3g} {}".format(sigpart, suffix)
def get_algorithm(path):
    algs = ["rkde", "ic2", "sklearn", "nocut", "simple"]
    for alg in algs:
        if alg in path:
            return alg
def get_dataset(path):
    datasets = {
        "bgauss": "gauss",
        "hep": "hep",
        "tmy3_4": "tmy3_4",
        "tmy3.": "tmy3",
        "mnist_64": "mnist",
        "home": "home",
    }
    for dataset in datasets.keys():
        if dataset in path:
            return datasets[dataset]

In [4]:
df = pd.read_csv("../experiments/d_raiders/scale_d.csv")

In [5]:
df

,algorithm,dataset,dim,num_kernels,num_test,num_train,out_path,test_time,train_time
0,ic2,bigdata/pmnist.csv,128,13761047.0,497,70000,./ic2/out/mnist_128.out,20027.000000,245429.000000
1,ic2,bigdata/pmnist.csv,4,161574342.0,97876,70000,./ic2/out/mnist_4.out,20001.000000,1963.000000
2,ic2,bigdata/pmnist.csv,64,23286200.0,1185,70000,./ic2/out/mnist_64.out,20010.000000,77559.000000
3,ic2,bigdata/pmnist.csv,32,54142113.0,2664,70000,./ic2/out/mnist_32.out,20002.000000,37366.000000
4,ic2,bigdata/pmnist.csv,2,261528485.0,1215929,70000,./ic2/out/mnist_2.out,20001.000000,712.000000
5,ic2,bigdata/pmnist.csv,1,291987663.0,6977570,70000,./ic2/out/mnist_1.out,20001.000000,477.000000
6,ic2,bigdata/pmnist.csv,16,82687978.0,4947,70000,./ic2/out/mnist_16.out,20003.000000,18618.000000
7,ic2,bigdata/pmnist.csv,8,123788131.0,11921,70000,./ic2/out/mnist_8.out,20001.000000,8855.000000
8,ic2,bigdata/pmnist.csv,256,9538099.0,316,70000,./ic2/out/mnist_256.out,20029.000000,679610.000000
9,sklearn,bigdata/pmnist.csv,128,NaN,3000,70000,./sklearn/out/mnist_128.out,102243.639469,741.921902


In [6]:
df["r_algorithm"] = df["out_path"].map(get_algorithm)
df["r_dataset"] = df["out_path"].map(get_dataset)
df["throughput"] = (
    1000 * df["num_test"] / df["test_time"]
)
df["a_throughput"] = (
    1000 * df["num_train"] 
    / (df["test_time"]*df["num_train"]/df["num_test"] + df["train_time"])
)

In [7]:
df = df.sort_values(by=["r_algorithm", "dim"])

In [8]:
algorder = ["ic2", "simple", "sklearn"]

In [11]:
fig = plt.figure(figsize=(6,4),dpi=300)
ax = plt.subplot(111)
for alg in algorder:
    cur_df = df[df["r_algorithm"] == alg]
    ax.loglog(
        cur_df["dim"],
        cur_df["throughput"],
        marker="o",
        label=alg)
ax.set_xlabel("Dataset Dimension")
ax.set_ylabel("Query Throughput")
ax.set_title("Adjusting Dimension, mnist, n={}".format(
        human_format(df.num_train.iloc[0]),
        df.dim.iloc[0]))
ax.legend(loc=0)
fig.tight_layout()
fig.savefig("d_scale.pdf")
fig.clear()

/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
